# Financial Applications of Reinforcement Learning

Advancements in Reinforcement Learning can be applied to a financial setting. In this subsection, two financial applications are introduced. The first application applies the G-learning algorithm, described in the previous section, in a goal-based wealth management environment. The second application presents a model proposed for optimal consumption, life insurance and investment with a terminal PDE as the solution. This model can be extended in multiple ways ,and can be applied in a broad range of financial scenarios, making it an attractive model for consumers. The only problem of the model is the curse of dimensionality which makes the Hamilton-Jacobi-Bellman (HJB) equation of the model in higher dimensions impossible to solve. The deep BSDE method, presented in the previous section, can be applied to the problem, thereby solving the HJB equation in higher dimensions. Although the method originates from optimal control literature, it resembles many features of the reinforcement algorithms.  

## G-learner for goal-based retirment plan optimization

Before the algorithm is applied in the retirement plan optimization setting, the G-learner needs to be described to the terminal setting. Recall from the G-learner section that the action free-energy function is  
```{math}
:label: H-RPO
H^{\pi}(s,a) = \mathbb{E}[R|s,a] + \gamma \mathbb{E}[F^{\pi}(s')|s,a] 
```

where
```{math}
:label: F-RPO
F^{\pi}(s) = \sum_a \pi(a|s)[\frac{1}{\beta}log\frac{\pi(a|s)}{\pi_0(a|s)} + H^{\pi}(s,a)]
```


The optimal action policy could be derived from $F^{\pi}(s)$ by maximizing the funcion by {eq}`soft-min` and then evaluating $F^{\pi}(s)$ at that point {eq}`soft-minF`. Using {eq}`soft-minF` and applying it in {eq}`soft-min` gives us the optimal action policy 
```{math}
:label: pi-RPO
\pi(a_t|s_t) = \pi_0(a_t|s_t)e^{\beta (H^{\pi}_t(s_t,a_t) - F_t^{\pi}(s_t) )} 
```
These three equation {eq}`H-RPO`, {eq}`F-RPO` and {eq}`pi-RPO` create a system of equations for G-learning that should be solved self-consistently for $\pi(a_t|s_t)$, $H^{\pi}_t(s_t,a_t)$ and $F^{\pi}_t(s_t)$ by backward recursion for $t = T-1, ..., 0$ with terminal conditions 

$$ H^{\pi}_T(s_t, a_T^*) = \hat{R}T(s_t,a_T^*)$$
$$ F_t^{\pi} = G_T^{\pi}(s_t,a_T^*) = \hat{R}_T(s_t,a_T^*) $$

The system of equations can be reduced to a non-linear equation when the rewards are observed. 

$$ G^{\pi}_t(s,a) = \hat{R}(s_t,a_t) + \mathbb{E}_{t,a}\left[\frac{\gamma}{\beta} log \sum_{a_{t+1}} \pi_0(a_{t+1}|x_{t+1})e^{\beta H^{\pi}_{t+1}(x_{t+1}, a_{t+1})}\right] $$

This is the soft relaxation of the Bellman optimality equation for the action-value Q-function, described in the G-learner section. 

Before the G-learning algorithm can be applied, the signal process of the environment should be presented. Therefore, we build in the first subsection a pre-specified target for our portfolio optimization together with the return specifications. The algorithm can then optimize its behavior by adapting to the target. 


### The target of the investment profolio 

Following the assumptions and notation in the model of {cite}`dixon2020g` for the investment portfolio. The model can be described as follows. the position in the assets $i = 1, 2, ..., n$ of the portfolio model are set in dollar values denoted as a vector $s_t$, with components $(s_t)_i$ for a dollar value of asset $i$ at the beginning of period $t$. The first asset $n=1$ of the portfolio is a risk-free bank account with risk-free interest rate $r_f$, while the other assets are risky with uncertain returns $r_t$, whose expected values are $\overline{r}_t$. The covariance matrix of return is $\Sigma_r$ of size $(N-1) \times  (N-1)$. $u_t$ are the trades of the investement portfolio at timestep $t$, while $c_t$ is the cash installment in the plan at time $t$. Both $u_t$ and $c_t$ need to be optimized and this pair $(c_t, u_t)$ can thus be considered the action variables of the dynamic optimization problem corresponding to the retirement plan. 

A pre-specified target value $\hat{P}_{t+1}$ is set at time $t$ for the next time step $t+1$. The target value $\hat{P}_{t+1}$ is set such that at step $t$ it will exceed the next step value $V_{t+1} = (1+ r_t)(s_t +u_t)$ of the portfolio. The reward system of the environment is determined such that a penalty is created when the portfolio performance is under-cutting the target value. 

```{math}
:label: target
R_t(s_t, u_t, c_t) = - c_t - \lambda \mathbb{E}_t \left[\left(\hat{P}_{t+1} - (1+r_t)(s_t + u_t)\right)_+\right] - u_t^{\top} \Omega u_t
```

The first term is the installment amount $c_t$ at the beginning of time period $t$, the second is the penalty if the portfolio underperformance the target value ,and the last term is an approximation for transaction costs using a convex function with parameter matrix $\Omega$ and serves as a $L_2$ regularization (see next section). 

We modify {eq}`target` in two ways for two reasons. First, the two decision vriabales $c_t$ and $u_t$ are not independent and have the following constraint:

$$ \sum_{n=1}^N = c_t $$

Thereby, setting the total change in all positions equal to the cash installments at time $t$.  The second reason is that the $max(\centerdot,0)$ operator is difficult to work with under the expectation and is therefore approximated with a quadratic function. This gives us   

```{math}
:label: adapted-target
 R_t(s_t, u_t) = - \sum_{n=1}^N u_{tn} - \lambda \mathbb{E}_t \left[\left(\hat{P}_{t+1} - (1+r_t)(s_t+u_t)\right)^2\right] -u_t^{\top} \Omega u_t 
```

The benefits of the adapted value function are twofold. First, it is able to solve the constraint between the cash injection $c_t$ and the trades $u_t$, reducing the dimensionality of the optimization problem. The second benefit is making the reward function highly tractable by transforming it into a quadratic function in actions $u_t$. The only drawback is that the penalization is symmetric, penalizing both $V_{t+1} \gg \hat{P}_{t+1}$ and $V_{t+1} \ll \hat{P}_{t+1}$. To mitigate the drawback, we only consider target values considerably higher than the expectation of the next-period portfolio value. A good choice for the target value $\hat{P}_{t+1}$ can for example be a linear combination of a portfolio-independent benchmark $B_t$ and a portfolio fixed-rate growth $\eta$: 

$$ \hat{P}_{t+1} = (1-\rho)B_t + \rho \eta \mathbb{1}^{\top}s_t $$

Where $0 \leq \rho \leq 1$ is the relative weight of the two terms and $\eta >1 $ defines the desired growth rate of the current portfolio. Note, that $B_t$ and $\eta$ need to be sufficiently large such that {eq}`adapted-target` is a reasonable proxy of {eq}`target`. The advantage of the target portfolio is that both parameters can be learned from an observed behavior of a financial agent using Inverse Reinforcement Learning. 

Equation {eq}`adapted-target` can be written in a quadratic form, once we denote the return as $r_t =\overline{r}_t + \tilde{\epsilon}_t$ where the first component is the risk-free rate and  where $\tilde{\epsilon}_t$ is an idiosyncratic noise with covariance $\Sigma_r$ of size $(N-1) \times (N-1)$. 

```{math}
:label: quadreturn
    R_t(s_t, u_t) & = - \lambda \hat{P}^2_{t+1} - u^{\top}_t \mathbb{1} + 2\lambda \hat{P}_{t+1}(s_t + u_t)^{\top}(1+\overline{r}_t) - \lambda(s_t + u_t)^{\top} \hat{\Sigma}_t(s_t + u_t) - u_t^{\top} \Omega u_t 
\\
    & = s^{\top}_t R_t^{(ss)}s_t + u_t^{\top}R_t^{(us)}s_t + u_t^{\top}R_t^{(uu)}u_y + s_t^{\top} R_t^{(s)} + u_t^{\top}R_t^{(u)} + R_t^{(0)} 
```

where 

\begin{equation}
    \begin{split}
         & \hat{\Sigma}_t = \begin{bmatrix} 
                    0 & 0\\
                    0 & \Sigma 
                    \end{bmatrix}  + ( 1+ \overline{r}_t)(1+\overline{r}_t)^{\top}
\\
         & R_t^{(ss)} = - \lambda \eta^2 \rho^2 \mathbb{1} \mathbb{1}^{\top} + 2\lambda \eta \rho(\mathbb{1} + \overline{r}_t) \mathbb{1}^{\top} - \lambda \hat{\Sigma}_t 
\\ 
        & R_t^{(us)} = 2\lambda \eta \rho (1+\overline{r}_t)\mathbb{1}^{\top} - 2\lambda \hat{\Sigma}_t 
\\ 
        & R_t^{(uu)} = -\lambda \hat{\Sigma}_t - \Omega
\\ 
        & R_t^{(s)} = -2\lambda \eta \rho (1-\rho)B_t\mathbb{1} + 2\lambda(1- \rho)B_t(1+\overline{r}_t) 
\\ 
        & R_t^{u} = -\mathbb{1} + 2 \lambda(1-\rho)B_t(1-\overline{r}_t)
\\ 
        & R_t^{(0)} = -(1-\rho)^2\lambda B^2_t
    \end{split}
\end{equation}

The free parameters defining the reward function are thus $\lambda$, $\eta$, $\rho$ and $\Omega$

### Applying the G-learning to the investment portfolio

A semi-analytical formulation of the G-learning is applied following Dixen et al. {cite}`dixon2020g`. First, a functional form of the value function as a quadratic form if $s_t$ is applied: 
```{math}
:label: quadvalue
F_t^{\pi}(s_t) = s_t^{\top} F_t^{(ss)}s_t + s_t^{\top}F_t^{(s)} + F_t^{(0)}
```

The dynamic equation is written as follows

```{math}
:label: dynamsyst
s_{t+1} = A_t(s_t+u_t) + (s_t + u_t) \circ \tilde{\epsilon}_t \text{,} \hspace{0.4cm} A_t = diag(1+\overline{r}_T) \text{,} \hspace{0.4cm} \tilde{\epsilon}_t = (0, \epsilon_t)
```

Where the expected return $\overline{r}_t$ are available as an output of a separate statistical model like a factor model framework. The coefficients of {eq}`quadvalue` are computed backward in time starting from the last maturity $t = T-1$. At time-step $t= T-1$, the reward function {eq}`quadvalue` can be optimized using an analytical approach by the following action: 

```{math}
:label: optimized
u_{T-1} = \tilde{\Sigma}_{T-1}^{-1}\left(\frac{1}{2\lambda}R_t^{(u)} + \frac{1}{2\lambda}R_t^{(us)}x_{T-1}\right)
```

where $\tilde{\Sigma}_{T-1}$ is defined as 

```{math}
:label: optimizedcov
 \tilde{\Sigma}_{T-1} = \hat{\Sigma}_{T-1} + \frac{1}{\lambda}\Omega 
 ``` 

Notice that the last term $\Omega$ which represents the convex costs in {eq}`target`, creates an $L_2$ regularization of matrix inversion in {eq}`optimizedcov`.

Now that the reward function is solved for time-step $t= T-1 $, the coeffiecients $F_{T-1}^{(ss)}$, $F_{T-1}^{(s)}$ and $F_{T-1}^{(0)}$ of the value function  can be calculated. We know that at time-step $t = T-1$, the value function needs to equal the reward function $ F_{T-1}^{\pi}(s_{T-1})= \hat{R}_{T-1}$. Indeed,  by plugging {eq}`optimized` back into {eq}`quadreturn` and comparing the result with {eq}`quadvalue`, we get the following terminal conditions for the parameters of {eq}`quadvalue`: 

$$ F_{T-1}^{(ss)} = R_{T-1}^{(ss)} + \frac{1}{2\lambda}[R_{T-1}^{(us)}]^{\top} [\tilde{\Sigma}_{T-1}^{-1}]^{\top}R_{T-1}^{(us)} + \frac{1}{4\lambda^2}[R_{T-1}^{(us)}]^{\top}[\tilde{\Sigma}_{T-1}^{-1}]^{\top} R_{T-1}^{(uu)}\tilde{\Sigma}_{T-1}^{-1}R_{T-1}^{(us)} $$

$$ F_{T-1}^{(s)} = R_{T-1}^{(s)} + \frac{1}{\lambda}[R_{T-1}^{(us)}]^{\top} [\tilde{\Sigma}_{T-1}^{-1}]^{\top}R_{T-1}^{(u)} + \frac{1}{2\lambda^2}[R_{T-1}^{(us)}]^{\top}[\tilde{\Sigma}_{T-1}^{-1}]^{\top} R_{T-1}^{(uu)}\tilde{\Sigma}_{T-1}^{-1}R_{T-1}^{(u)} $$

$$ F_{T-1}^{(0)} = R_{T-1}^{(0)} + \frac{1}{2\lambda}[R_{T-1}^{(u)}]^{\top} [\tilde{\Sigma}_{T-1}^{-1}]^{\top}R_{T-1}^{(u)} + \frac{1}{4\lambda^2}[R_{T-1}^{(u)}]^{\top}[\tilde{\Sigma}_{T-1}^{-1}]^{\top} R_{T-1}^{(uu)}\tilde{\Sigma}_{T-1}^{-1}R_{T-1}^{(u)}$$ 


Any other time-step $t =T-2,..., 0$ is computed using backward recursion following the Bellman equation but first, the conditional expectation of the next period F-function is computed using {eq}`dynamsyst`. 

$$ \mathbb{E}_{t,a}[F^{\pi}_{t+1}(s_{t+1})] = (s_t + u_t)^{\top} \left( A^{\top}_t \overline{F}_{t+1}^{(ss)}A_t + \tilde{\Sigma}_r \circ \overline{F}^{(ss)}_{t+1} \right) (s_t + u_t) \\ 
+ (s_t + u_t)^{\top} A^{\top}_t \overline{F}_{t+1}^{(s)} + \overline{F}_{t+1}^{(0)}, \hspace{0.4cm} \tilde{\Sigma}_r = 
\begin{bmatrix} 
0 & 0 \\ 
0 & \Sigma_r 
\end{bmatrix}  $$ 

where $\overline{F}_{t+1}^{ss} = \mathbb{E}[F_{t+1}^{(ss)}]$ similarly for $\overline{F}_{t+1}^{(x)}$ and  $\overline{F}_{t+1}^{(0)}$. Now, we plug in the conditional expectation of the F-function and the reward function in the Bellman equation

$$ H_t^{\pi}(s_t, u_t) = \hat{R}_t(s_t, u_t) + \gamma \mathbb{E}_{t,u}[F_{t+1}^{\pi}(s_{t+1}|s_t,u_t)] $$ 

Notice that both the reward function {eq}`quadreturn` and the conditional expectation of the F-function are quadratic functions of $x_t$ and $u_t$ in the Bellman equation. This means that the action-value function is also a quadratic function  of $x_t$ and $u_t$: 

```{math}
:label: action-valueG
H_t^{\pi}(s_t, u_t) = s_t^{\top} Q^{(ss)}_t s_t + u_t^{\top} Q_t^{(us)}s_t + u^{\top}_t Q_t^{(uu)}u_t + s_t^{\top}Q_t^{(s)} + u^{\top}_t Q^{(u)}_t + Q_t^{(0)} 
```

where 

\begin{equation}
    \begin{split}
        & Q^{(ss)}_t = R_t^{(ss)} + \gamma(A_t^{\top}\overline{F}_{t+1}^{(ss)} A_t + \tilde{\Sigma}_r \circ \overline{F}_{t+1}^{ss})
\\ 
        & Q^{(us)}_t = R_t^{(us)} + 2\gamma(A_t^{\top}\overline{F}_{t+1}^{(ss)} A_t + \tilde{\Sigma}_r \circ \overline{F}_{t+1}^{ss}) 
\\ 
        & Q^{(uu)}_t = R_t^{(uu)} + \gamma(A_t^{\top}\overline{F}_{t+1}^{(ss)} A_t + \tilde{\Sigma}_r \circ \overline{F}_{t+1}^{ss}) - \Omega 
\\ 
        & Q^{(s)}_t = R_t^{(s)} + \gamma A_t^{\top}\overline{F}_{t+1}^{(s)}
\\
        & Q^{(u)}_t = R_t^{(u)} + \gamma A_t^{\top}\overline{F}_{t+1}^{(s)}
\\
        & Q^{(0)}_t = R_t^{(0)} + \gamma F_{t+1}^{(0)}
    \end{split}
\end{equation}

Now that the action-value function is computed, we only need to calculate the F-function for the current step. Remember from {eq}`soft-minF` that the F-function can be described by the prior policy distribution $\pi_0(u_t|x_t)$  and the action value function $H_t^{\pi}(s_t, u_t)$ .

$$ F_t^{\pi}(s_t) = \frac{1}{\beta}log \int \pi_0(u_t|s_t)e^{\beta H_t^{\pi}(s_t, u_t)}du_t $$ 
 
Let the prior policy be a Gaussian: 

$$ \pi_0(u_t|s_t) = \frac{1}{\sqrt{(2\pi)^n|\Sigma_p|}}e^{-\frac{1}{2}(u_t - \hat{u}_t)^{\top}\Sigma_p^{-1}(u_t - \hat{u}_t)} $$ 

Where the mean value is a linear function of the state $x_t$. 


$$ \hat{u}_t = \overline{u}_t + \overline{v}_t s_t$$ 

By applying the n-dimesnional Gaussian integration formula, the integration over $u_t$ in {eq}`` can be performed analytically 

$$ \int e^{\frac{1}{2}u^{\top}Au+u^{\top}B} d^{n}u = \sqrt{\frac{(2\pi)}{|A|}} e^{\frac{1}{2}B^{\top}A^{-1}B} $$

where $|A|$ denotes the determinant of matrix $A$. 


Once the Gaussian integration is performed, we can again compare the resulting expression with {eq}`quadvalue` and obtain the following coefficients: 


```{math}
:label: entropyvalue
    F^{\pi}_t(s_t) & = s_t^{\top}F_t^{(ss)}s_t + s_t^{\top}F_t^{s} + F^{0}_t 
\\ 
    F_t^{(ss)} & = Q_t^{(ss)} + \frac{1}{2\beta}(U^{\top}_t\overline{\Sigma}_p^{-1} U_t - \overline{v}_t^{\top}\Sigma_p^{-1} \overline{v}_t)
\\
    F_t^{(s)} & = Q_t^{(s)} + \frac{1}{\beta}(U^{\top}_t\overline{\Sigma}_p^{-1} W_t - \overline{v}_t^{\top}\Sigma_p^{-1} \overline{u}_t) 
\\
    F_t^{(0)} & = Q_t^{(0)} + \frac{1}{2\beta}(W^{\top}_t\overline{\Sigma}_p^{-1} W_t - \overline{u}_t^{\top}\Sigma_p^{-1} \overline{u}_t) - \frac{1}{2\beta}(log|\Sigma_p| + log|\overline{\Sigma_p}|) 
```

With auxiliary parameters 

$$ U_t = \beta Q^{(us)} + \Sigma^{-1}_p\overline{v}_t $$ 
$$ W_t = \beta Q_t^{(u)} + \Sigma^{-1}_p \overline{u}_t $$
$$ \overline{\Sigma}_p = \Sigma_p^{-1} - 2 \beta Q_t^{(uu)} $$ 

Now from {eq}`pi-RPO` we know that the optimal policy is 

$$ \pi(u_t|s_t) = \pi_0(u_t|s_t)e^{\beta(H^{\pi}_t(s_t,u_t) - F_t^{\pi}(s_t))} $$

By applying the action-value function $G^{\pi}_t(x_t,u_T)$ and the entropy regularized value function {eq}`entropyvalue` to the optimal policy, we obtain the update of the policy parameters for time-step $t$. 

```{math}
:label: updateG
 \pi(u_t|s_t) = \frac{1}{\sqrt{(2 \pi)^n |\tilde{\Sigma}_p|}} e^{-\frac{1}{2}(u_t - \tilde{u}_t - \tilde{v}_t s_t)^{\top} \Sigma_p^{-1}(u_t - \hat{u}_t - \tilde{v}_t s_t)}
```

where

$$\tilde{\Sigma}^{-1}_p = \Sigma_p^{-1} -2\beta Q^{(uu)} $$ 
$$ \tilde{u}_t = \tilde{\Sigma}_p (\Sigma^{-1}_p \overline{u}_t + \beta Q^{(u)}_t) $$ 
$$ \tilde{v}_t = \tilde{\Sigma}_p (\Sigma_p^{-1}\overline{v}_t + \beta Q_t^{(ux)}) $$ 

the G-learning algorithm is like any other RL algorithm and follows the GLI. The algorithm has thus a policy evaluation and a policy improvement process. Namely, equation  {eq}`action-valueG`and {eq}`entropyvalue` evaluate the current policy, while {eq}`updateG`  improves the policy. These processes work in tandem at each time step $t$, until the convergence criterium is met. Note, that there is an additional step to calculate the optimal contribution at each time step, because of the budget constraint developed in the subsection of building the target. By simply adding each optimal action $u_t$ at time step $t$ we can obtain the optimal cash contribution. A numerical example of the G-learner algorithm for a goal-based retirement plan can be found in {cite}`DixonMatthewF2020MLiF`.  

## Reinforcement learning in optimal control: applying the Deep BSDE method

For the Deep BSDE method to be useful in financial scenarios, a financial model for which the solution is a terminal PDE should first be derived. In this chapter, we introduce such a model called the optimal consumption, investment ,and life insurance model. Before applying the model following ye {cite}`ye2006optimal`, an introduction to the model is given together with various extensions of Ye formulation. 

### an Optimal consumption, investment and life insurance model

The first person to include uncertain lifetime and life insurance decisions in a discrete life-cycle model was Yaari {cite}`yaari1965uncertain`. He explored the model using a utility function without bequest (Fisher Utility function) and a utility function with bequest (Marshall Utility function) in a bounded lifetime. In both cases, he looked at the implications of including life insurance. Although Yaari's model was revolutionary in the sense that now the uncertainty of life could be modeled, Leung {cite}`leung1994uncertain` found that the constraints laid upon the Fisher utility function were not adequate and lead to terminal wealth depletion. Richard {cite}`richard1975optimal` applied the methodology of Merton {cite}`merton1969lifetime, merton1975optimum` to the problem setting of Yaari in a continuous time frame. Unfortunately, Richard's model had one deficiency: The bounded lifetime is incompatible with the dynamic programming approach used in Merton's model. As an individual approaches his maximal possible lifetime T, he will be inclined to buy an infinite amount of life insurance. To circumvent this Richard used an artificial condition on the terminal value. But due to the recursive nature of dynamic programming, modifying the last value would imply modifying the whole result. Ye {cite}`ye2006optimal`  found a solution to the problem by abandoning the bounded random lifetime and replacing it with a random variable taking values in $[0,\infty)$. The models that replaced the bounded lifetime, are thereafter called intertemporal models as the models did not consider the whole lifetime of an individual but rather looked at the planning horizon of the consumer.  Note that the general setting of Ye {cite}`ye2006optimal` has a wide range of theoretical variables, while still upholding a flexible approach to different financial settings. On this account, it is a good baseline to confront the issues concerning the current models of financial planning. 

After Ye {cite}`ye2006optimal` various models have been proposed which all have given rise to unique solutions to the consumption, investment, and insurance problem. The first unique setting is a model with multiple agents involved. For example,  Bruhn and Steffensen {cite}`bruhn2011household` analyzed the optimization problem for couples with correlated lifetimes with their partner nominated as their beneficiary using a copula and common-shock model, while Wei et al.{cite}`wei2020optimal` studied optimization strategies for a household with economically and probabilistically dependent persons. Another setting is where certain constraints are used to better describe the financial situation of consumers. Namely, Kronborg and Steffensen {cite}`kronborg2015optimal` discussed two constraints. One constraint is a capital constraint on the savings in which savings cannot drop below zero. The other constrain involves a minimum return in savings. A third setting describes models that analyze the financial market and insurance market in a pragmatic environment. A good illustration is the study of Shen and Wei {cite}`shen2016optimal`. They incorporate all stochastic processes involved in the investment and insurance market where all randomness is described by a Brownian motion filtration. An interesting body of models is involved in time-inconsistent preferences. In this framework, consumers do not have a time-consistent rate of preference as assumed in the economic literature. There exists rather a divergence between earlier intentions and later choices De-Paz et al. {cite}`de2014consumption`. This concept is predominantly described in psychology. Specifically, rewards presented closer to the present are discounted proportionally less than rewards further into the future. An application of time-inconsistent preferences in the consumption, investment, and insurance optimization can be found in Chen and Li {cite}`chen2020time` and De-Paz et al. {cite}`de2014consumption`. 


### The model specifications

In this section, I will set the dynamics for the baseline model in place. The dynamics follow primarily from the paper of Ye {cite}`ye2006optimal`.

Let the state of the economy be represented by a standard Brownian motion $w(t)$, the state of the consumer's wealth be characterized by a finite state multi-dimensional continuous-time Markov chain $X(t)$ and let the time of death be defined by a non-negative random variable $\tau$. All are defined on a given probability space ($\Omega, \mathcal{F}, \mathbb{P} $) and $W(t)$ is independent of $\tau$. Let $T< \infty$ be a fixed planning horizon. This can be seen as the end of the working life for the consumer. $\mathbb{F} = \{\mathcal{F}_t, t \in [0,T]\}$, be the P-augmentation of the filtration $\sigma${$W(s), s<t \}, \forall t \in [0,T]$ , so $\mathcal{F}_t$ represents the information at time t. The economy consist of a financial market and an insurance market. In the following section I will construct these markets seperately following Ye {cite}`ye2006optimal`. 

The financial market consist of a risk-free security $B(t)$ and a risky security $S(t)$, who evolve according to 

$$ \frac{dB(t)}{B(t)}=r(t)dt $$

$$ \frac{dS(t)}{S(t)}=\mu(t)dt+\sigma(t)dW(t)$$

Where $\mu, \sigma, r > 0$ are constants and $\mu(t), r(t), \sigma(t): [0,T] \to R$ are continuous. With $\sigma(t)$ satisfying $\sigma^2(t) \ge k, \forall t \in [0,T]$

The random variable $\tau_d$ needs to be first modeled for the insurance  market. Assume that $\tau$ has a probability density function $f(t)$ and probability distribution function given by 

$$ F(t) \triangleq P(\tau < t) = \int_0^t f(u) du $$

Assuming $\tau$ is independent of the filtration $\mathbb{F}$ 

Following on the probability distribution function we can define the survival function as follows

$$ \bar{F}(t)\triangleq P(\tau \ge t) = 1 -F(t) $$

The hazard function is the  instantaneous death rate for the consumer at time t and is defined by 

$$ \lambda(t) = \lim_{\Delta t\to 0} = \frac{P(t\le\tau < t+\Delta t| \tau \ge t)}{\Delta t} $$

where $\lambda(t): [0,\infty[ \to R^+$ is a continuous, deterministic function with $\int_0^\infty \lambda(t) dt = \infty$.

Subsequently, the survival and probability density function can be characterized by 


$$ \bar{F}(t)= {}_tp_0= e^{-\int_0^t \lambda(u)du} $$
$$ f(t)=\lambda(t) e^{-\int_0^t\lambda(u)du} $$ 

With conditional probability described as 

$$ f(s,t) \triangleq \frac{f(s)}{\bar{F}(t)}=\lambda(s) e^{-\int_t^s\lambda(u)dy} $$
$$ \bar{F}(s,t) = {}_sp_t \triangleq \frac{\bar{F}(s)}{\bar{F}(t)} = e^{-\int_t^s \lambda(u)du} $$

    
Now that $\tau$ has been modeled, the life insurance market can be constructed. Let's assume that life insurance is continuously offered and that it provides coverage for an infinitesimally small period of time. In return, the consumer pays a premium rate p when he enters into a life insurance contract so that he might insure his future income. In compensation, he will receive a total benefit of $\frac{p}{\eta(t)}$ when he dies at time t. Where $\eta : [0,T] \to R^+ $ is a continuous, deterministic function.

Both markets are now described and the wealth process $X(t)$ of the consumer can now be constructed. Given an initial wealth $x_0$, the consumer receives a certain amount of income $i(t)$ $\forall t \in [0,\tau \wedge T]$ and satisfying $\int_0^{\tau \wedge T} i(u)du < \infty$. He needs to choose at time t a certain premium rate $p(t)$, a certain consumption rate $c(t)$ and a certain amount of his wealth $\theta (t)$ that he invest into the risky asset $S(t)$. So given the processes $\theta$, c, p and i, there is a wealth process $X(t)$  $\forall t \in [0, \tau \wedge T] $ determined by 

$$ dX(t) = r(t)X(t) + \theta(t)[( \mu(t) - r(t))dt +\sigma(t)dW(t)] -c(t)dt -p(t)dt + i(t)dt,   \quad t \in [0,\tau \wedge T] $$

If $t=\tau$ then the consumer will receive the insured amount $\frac{p(t)}{\eta(t)}$. Given is wealth X(t) at time t his total legacy will be 

$$ Z(t) = X(t) + \frac{p(t)}{\eta(t)} $$ 

The predicament for the consumer is that he needs to choose the optimal rates for c, p , $\theta$ from the set $\mathcal{A}$ , called the set of admissible strategies, defined by 

$$ \mathcal{A}(x) \triangleq  \textrm{set of all possible triplets (c,p,}\theta) $$ 

such that his expected utility from consumption, from legacy when $\tau > T$ and from terminal wealth when $\tau \leq T $  is maximized. 

$$ V(x) \triangleq \sup_{(c,p,\theta) \in \mathcal{A}(x)} E\left[\int_0^{T \wedge \tau} U(c(s),s)ds + B(Z(\tau),\tau)1_{\{\tau \ge T\}} + L(X(T))1_{\{\tau>T\}}\right] $$ 

Where $U(c,t)$ is the utility function of consumption, $B(Z,t)$ is the utility function of legacy ,and $L(X)$ is the utility function for the terminal wealth. $V(x)$ is called the value function and the consumers wants to maximize his value function by choosing the optimal set $\mathcal{A} = (c,p,\theta)$. The optimal set $\mathcal{A}$ is found by using the dynamic programming technique described in the following section. 

### dynamic programming principle 

To solve the consumer's problem the value function needs to be restated in a dynamic programming form. 

$$J(t, x; c, p, \theta) \triangleq E \left[\int_0^{T \wedge \tau} U(c(s),s)ds + B(Z(\tau),\tau)1_{\{\tau \ge T\}} + L(X(T))1_{\{\tau>T\}}| \tau> t, \mathcal{F}_t \right] $$

The value function becomes

$$ V(t,x) \triangleq \sup_{\{c,p,\theta\} \in \mathcal{A}(t,x)} J(t, x; c, p, \theta)  $$

Because $\tau$ is independent of the filtration, the value function can be rewritten as 

$$ E \left[\int_0^T  \bar{F}(s,t)U(c(s),s) + f(s,t)B(Z(\tau),\tau) ds  + \bar{F}(T,t)L(X(T))| \mathcal{F}_t \right]$$ 

The optimization problem is now converted from a random closing time point to a fixed closing time point. The mortality rate can also be seen as a discounting function for the consumer as he would value the utility on the probability of survival. 

Following the dynamic programming principle, we can rewrite this equation as the value function at time s plus the value created from time step t to time step s. This enables us to view the optimization problem into a time step setting, giving us the incremental value gained at each point in time.   

$$ V(t,x) = \sup_{\{c,p,\theta\} \in \mathcal{A}(t,x)} E\left[e^{-\int_t^s\lambda(v)dv}V(s,X(s)) + \int_t^s f(s,t)B(Z(s),s) + \bar{F}(s,t)U(c(s),s)ds|\mathcal{F}_t\right] $$ 

The Hamiltonian-Jacobi-bellman (HJB) equation can be derived from the dynamic programming principle and is as follows

```{math}
:label: BELL2

\begin{cases} 
V_t(t,x) -\lambda V(t,x) + \sup_{(c,p,\theta)} \Psi(t,x;c,p,\theta)  = 0 \\ V(T,x) = L(x)  
\end{cases}

```

where 

$$ \Psi(t,x; c,p,\theta) \triangleq r(t)x + \theta(\mu(t) -r(t)) + i(t) -c -p)V_x(t,x) + \\ \frac{1}{2}\sigma^2(t)\theta^2V_{xx}(t,x) + \lambda(t)B(x+ p/\eta(t),t) + U(c,t) $$


Proofs for deriving the HJB equation, dynamic programming principle ,and converting from a random closing time point to a fixed closing time point can be found in Ye {cite}`ye2006optimal`

A strategy is optimal if  


\begin{equation}
    \begin{split}
        0 & =V_t(t,x) -\lambda(t)V(t,x) + \sup_{c,p,\theta}(t,x;c,p,\theta)  
\\
        0 & = V_t(t,x) -\lambda(t)V(t,x) + (r(t)x+ i(t))V_x + \sup_c\{U(c,t)-cV_x\} 
\\ 
        & + \sup_p\{\lambda(t)B(x + p/\eta(t),t) - pV_x\} + \sup_\theta \{ \frac{1}{2}\sigma^2(t)V_{xx}(t,x)\theta^2 +(\mu(t) - r(t))V_x(t,x)\theta\} 
    \end{split}
\end{equation}


The first order conditions for regular interior maximum are
```{math}
:label: cons_cond
    \sup_c  \{ U(c,t) - cV_x\} & = \Psi_c(t,x;c^*,p^*,\theta^*)  
\\ 
    \rightarrow  0 & = -V_x(t,x) + U_c(c*,t) 
\\
    \sup_p\{\lambda(t)B(x + p/\eta(t),t) - pV_x\} & = \Psi_p(t,x;c^*,p^*,\theta^*)
\\ 
    \rightarrow 0 & = -V_x(t,x) + \frac{\lambda(t)}{\eta{t}}B_Z(x + p^*/\eta(t),t)
\\
    \sup_\theta \{ \frac{1}{2}\sigma^2(t)V_{xx}(t,x)\theta^2 +(\mu(t) - r(t))V_x(t,x)\theta\} & = \Psi_\theta(t,x;c^*,p^*,\theta^*)
\\
     \rightarrow 0 & = (\mu(t) -r(t))V_x(t,x) + \sigma^2(t)\theta^*V_{xx}(t,x)
```

The second order conditions are 

$$ \Psi_{cc}, \Psi_{pp}, \Psi_{\theta \theta} < 0 $$ 


This optimal control problem has been solved analytically by Ye {cite}`ye2006optimal` for the Constant Relative Risk Aversion utility function. To solve {eq}`BELL2` the  BSDE method can be used. The Deep BSDE method was the first deep learning-based numerical algorithm to solve general nonlinear parabolic PDEs in high dimensions. 

remember the general form of PDEs which the Deep BSDE method solves: 

```{math}
:label: gen_form2
\frac{\partial u}{\partial t} + \frac{1}{2} Tr(\sigma \sigma^T (Hess_xu) + \Delta u(t,x)  \mu(t,x) + f(t,x,u, \sigma^T(\Delta_x u)) = 0 
``` 
With some terminal condition $u(T,x) = g(x)$. {eq}`BELL2` can thus be reformulated in the general form: 

$$\underbrace{V_t(t,x)}_{\frac{\partial u}{\partial t}} + \underbrace{\frac{1}{2}\sigma(t)^2\theta^2V_{xx}(t,x)}_{\frac{1}{2}Tr(\sigma \sigma^T(Hess_xu(t,x)))} + \underbrace{(r(t) x + \theta(\mu(t) -r(t)) + i(t) -c -p)V_x(t,x)}_{\Delta u(t,x)\mu (t,x)} \\ + \underbrace{\lambda(t)B(x+\frac{p}{\eta(t)},t) + U(t,x) - \lambda(t)V(t,x)}_{f(t,x,u(t,x), \sigma^T(t,x)\Delta u(t,x))}$$

The BSDE method can thus be applied. 